In [1]:
# To access spotify API
!pip install spotipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 5.1 MB/s eta 0:00:00


In [26]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [27]:
# Permissions to access data from spotify, we get client id and secret key when we create and app in developers spotify
Client_credentials_manager = SpotifyClientCredentials(client_id='b8d0772b68ba48d9931534d43d06f164',client_secret='d74687e13fa94c8fbba49d52c1f7cfd2')

In [28]:
#create spotify object to extract data using client credentials
sp = spotipy.Spotify(client_credentials_manager = Client_credentials_manager)

In [29]:
# To extract top 100 trending songs from spotify
playlist_link = "https://open.spotify.com/playlist/5ABHKGoOzxkaa28ttQV9sE?si=db978186f83a46c7"

In [30]:
#we want id of the playlist which is present before question mark in above url, we will extract that from the link.
playlist_URI = playlist_link.split("/")[-1].split('?')[0]

In [31]:
#this will give all the information about tracks(such as name of album, artist, total tracks, release date, year) under that URI in json format
data = sp.playlist_tracks(playlist_URI)

In [32]:
len(data['items'])

100

In [33]:
data['items'][0]['track']['album']['id']

'4yP0hdKOZPNshxUOjY0cZj'

In [16]:
data['items'][0]['track']['album']['name']

'After Hours'

In [17]:
data['items'][0]['track']['album']['release_date']

'2020-03-20'

In [18]:
data['items'][0]['track']['album']['total_tracks']

14

In [19]:
data['items'][0]['track']['album']['external_urls']['spotify']

'https://open.spotify.com/album/4yP0hdKOZPNshxUOjY0cZj'

In [38]:
# we will extract above data for all of the items in the above url
album_list = []
for row in data['items']:
  album_id = row['track']['album']['id']
  album_name = row['track']['album']['name']
  album_release_date = row['track']['album']['release_date']
  album_total_tracks = row['track']['album']['total_tracks']
  album_external_url = row['track']['album']['external_urls']['spotify']
  album_elements ={'album_id' : album_id, 'album_name' : album_name,
                   'album_release_date' : album_release_date, 'album_total_tracks':album_total_tracks,'album_external_url':album_external_url}
  album_list.append(album_elements)

In [47]:
#Here there can be multiple artists in an album, we need to capture that information as well.
data['items'][3]['track']['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/246dkjvS1zLTtiykXe5h60'},
  'href': 'https://api.spotify.com/v1/artists/246dkjvS1zLTtiykXe5h60',
  'id': '246dkjvS1zLTtiykXe5h60',
  'name': 'Post Malone',
  'type': 'artist',
  'uri': 'spotify:artist:246dkjvS1zLTtiykXe5h60'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/1zNqQNIdeOUZHb8zbZRFMX'},
  'href': 'https://api.spotify.com/v1/artists/1zNqQNIdeOUZHb8zbZRFMX',
  'id': '1zNqQNIdeOUZHb8zbZRFMX',
  'name': 'Swae Lee',
  'type': 'artist',
  'uri': 'spotify:artist:1zNqQNIdeOUZHb8zbZRFMX'}]

In [49]:
# we will extract artist related information
artist_list = []
for row in data['items']:
  for key,value in row.items():
    if key == "track":
      for artist in value['artists']:
        artist_list.append({'artist_id':artist['id'],'artist_name':artist['name'],'external_url':artist['href']})

In [53]:
# we will extract songs related information
songs_list = []
for row in data['items']:
  song_id = row['track']['id']
  song_name = row['track']['name']
  song_duration = row['track']['duration_ms']
  song_url = row['track']['external_urls']['spotify']
  song_popularity = row['track']['popularity']
  song_added = row['added_at']
  album_id = row['track']['album']['id']
  artist_id= row['track']['album']['artists'][0]['id']
  song_elements ={'song_id':song_id,'song_name':song_name,'song_duration':song_duration,'song_url':song_url,'song_popularity':song_popularity,'song_added':song_added,
                  'album_id' : album_id, 'artist_id' : artist_id,}
  songs_list.append(song_elements)

In [55]:
#converting dictionary list to dataframe
df_album = pd.DataFrame(album_list)
df_artist = pd.DataFrame(artist_list)
df_songs = pd.DataFrame(songs_list)

In [60]:
# Dropping duplicates and converting datetime field
df_album = df_album.drop_duplicates()
df_artist = df_artist.drop_duplicates()
df_songs = df_songs.drop_duplicates()

df_album['album_release_date'] = pd.to_datetime(df_album['album_release_date'])
df_songs['song_added'] = pd.to_datetime(df_songs['song_added'])

In [63]:
df_album.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 94 entries, 0 to 99
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   album_id            94 non-null     object        
 1   album_name          94 non-null     object        
 2   album_release_date  94 non-null     datetime64[ns]
 3   album_total_tracks  94 non-null     int64         
 4   album_external_url  94 non-null     object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 4.4+ KB
